# Fórmula Mágica - Joel Greenblat

Joel Greenblatt escreveu um livro (“The Little Book that Beats the Market”), onde explica como os investidores podem aplicar sistematicamente uma fórmula que busca bons negócios quando eles estão disponíveis a preços de pechincha. A essa estratégia deu o nome de Fórmula Mágica.

Essa fórmula usa dois indicadores:
* ROIC
* EV/EBIT

Baseado nesses indicadores queremos formar um ranking de papéis que possuam o maior EV/EBIT e o maior ROIC.

Essa é a forma bem simplificada, o próprio Greenblat faz algumas ressalvas e sugere mais alguns critérios para a escolha dos ativos como: 
* exclusão de bancos;
* empresas de infraestrura (utilities)
* empresas com capitalização de mercado inferior a US$ 50 milhões

Mas aqui, para efeito de estudos de desenvolvimento em Python vou me ater a apenas essas duas regras: encontrar os 20 ativos que estejam com o maior ROIC e EV/EBIT e exibir essa lista.
O formato dessa carteira é anual. Ou seja, o investidor compra as ações indicadas hoje e só vai trocar por novas ações no próximo ano.

#### 1. Buscar os dados no site Fundamentus

In [15]:
import yfinance
import requests
import pandas as pd
import numpy as np

url = "https://www.fundamentus.com.br/resultado.php"
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.127 Safari/537.36"}
r = requests.get(url, headers=headers)

lista = pd.read_html(r.text, decimal=",", thousands=".")
dados = lista[0]

#### 2. Tratar os dados

Nesta etapa vamos fazer um tratamento nos dados:
 
    * remover as colunas que não vamos utilizar e deixar apenas: Papel, EV/EBIT, ROIC;
    * remover as linhas que tenham EV/EBIT negativo;
    * remover as linhas com ROIC negativo;

In [16]:
#o site fundamentus também traz os outros indicadores sugeridos pelo Greenblat 
# ROIC – Quanto maior melhor EV/EBIT – Quanto menor melhor

#deletando todas as colunas que não serão utilizadas
dados.drop(columns=['Cotação', 
                    'P/VP', 
                    'PSR',
                    'P/L', 
                    'Div.Yield',
                    'P/Ativo',
                    'P/Cap.Giro',
                    'P/EBIT',
                    'P/Ativ Circ.Liq',
                    'EV/EBITDA',
                    'Mrg Ebit',
                    'Mrg. Líq.',
                    'Liq. Corr.',
                    'ROE',
                    'Patrim. Líq',
                    'Dív.Brut/ Patrim.',
                    'Cresc. Rec.5a'], inplace=True)

#tratamento dos dados para remover as distorções
#EV/EBIT negativo 
#indexNames = dados[dados['ROIC'] <= 0 ].index
#dados.drop(indexNames , inplace=True)
indexNames = dados[dados['EV/EBIT'] <= 0].index
dados.drop(indexNames , inplace=True)

#garantindo que os dados analisados estão em formato numérico
#transformando as porcentagens em valores numéricos
#não fui em quem criou essa função, mas achei muito boa e resolvi adicionar ao código
def columns_which_contains(df, value):
    """
    Serch for DataFrame column's values which contains a specific value
    :value: any characters, phrases, symbols
    :returns: list of DataFrame column's names
    """
    return [df[column].name for column in df if df[column].astype(str).str.contains(value).any()]

percentual_columns = columns_which_contains(dados, '\%')

for column_name in percentual_columns:
    dados[column_name] = dados[column_name].str.replace('.', '', regex=True)
    dados[column_name] = dados[column_name].str.replace(',', '.', regex=True)
    dados[column_name] = dados[column_name].str.replace('[%,]', '', regex=True).astype(float)
    dados[column_name] = dados[column_name]/100

#ROE negativo ou muito alto
#antes da tranformação dos dados do ROE o trecho abaixo dava erro
indexNames = dados[dados['ROIC'] <= 0 ].index
dados.drop(indexNames , inplace=True)
#indexNames = dados[dados['ROE'] >= 1 ].index
#dados.drop(indexNames , inplace=True)

#vou adicionar também um filtro de liquidez
#deixando apenas as ações com liquidez maior que 100000 nos últimos 2 meses
#seria bom checar qual o nível de liquidez aceitável para o investidor 
#conseguir entrar e sair do papelo sem problemas
indexNames = dados[dados['Liq.2meses'] < 100000].index
dados.drop(indexNames , inplace=True)


#### 3. Ranking

Agora, vamos construir um ranking das ações que se enquadram nos critérios.

A empresa com o maior ROIC recebe a nota 1, a segunda recebe nota 2 e assim por diante. A mesma coisa é feita para o EV/EBIT.

A partir dessa notas criaremos uma nova coluna com as notas SOMADAS dos dois indicadores. Aquelas com MENORES notas representam as “melhores” empresas pela combinação dos dois indicadores.

In [17]:
#ordenação dos dados de forma descendente do ROIC
dados.sort_values('ROIC', ascending=False, inplace=True)

#indexando os dados seguindo a ordenação descendente
#agora o maior ROIC está no índice 0 e o menor ROIC está no último índice
dados.index = range(0,len(dados))

dados["notaROIC"] = range(0,len(dados)) 

#ordenação dos dados de forma descendente do EV/EBIT
dados.sort_values('EV/EBIT', ascending=False, inplace=True)

dados["notaEV/EBIT"] = range(0,len(dados)) 


#fazendo a soma das notas

dados["somaNotas"] = 0

dfd = dados.copy()

for i in dfd.index:
    dfd.loc[i, 'somaNotas'] = dfd.loc [i, 'notaEV/EBIT']  + dfd.loc[i, 'notaROIC']

dfd.sort_values('somaNotas', ascending=True, inplace=True)

print("20 ativos - Fórmula Greenblat\n")
print(dfd.iloc[0:20])

20 ativos - Fórmula Greenblat

     Papel  EV/EBIT    ROIC   Liq.2meses  notaROIC  notaEV/EBIT  somaNotas
43   WEGE3    24.36  0.2425  269964000.0        43           28         71
33   TFCO4    16.25  0.2921    2505840.0        33           63         96
65   INTB3    22.82  0.1870   48898600.0        65           31         96
98   ARZZ3    21.92  0.1406   97282000.0        98           33        131
70   VIVA3    16.43  0.1733   24923200.0        70           61        131
18   ODPV3     8.16  0.3900   14120200.0        18          119        137
69   RENT3    14.48  0.1767  368685000.0        69           71        140
75   BOAS3    14.74  0.1682    9215890.0        75           68        143
16   PRIO3     7.91  0.3965  475940000.0        16          130        146
67   VITT3    12.70  0.1846    4782690.0        67           80        147
45   SOJA3     9.68  0.2396    6483100.0        45          103        148
3    ATOM3     6.81  0.6262     108794.0         3          146      

#### 4. Resultado

Esse é o método sistemático e sem influência de nenuma análise um pouco mais refinada. Nem mesmo eu me aprofundei nos indicadores usados na fórmula, se está tudo correto ou não (se alguém quiser contibuir e me corrigir sobre qualquer coisa, fique a vontade).

Será que é um bom método usar apenas o ROIC e o EV/EBIT para nos indicar a compra de ações?

Mas Greenblat criou essa Fórmula Mágica e será que ela não é rentável? Que tal um backtest de como seria o desempenho dessa carteira nos útimos anos?
Esse será o próximo desafio....

![Logo do Markdown](resultado.png)